In [1]:
import os, sys
dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: sys.path.append(dir1)

In [2]:
from brio.utils.Preprocessing import Preprocessing
from sklearn.model_selection import train_test_split
from pickle import dump, load
import pandas as pd
import numpy as np

from brio.bias.FreqVsFreqBiasDetector import FreqVsFreqBiasDetector

## Importing Data and Trained Classifier

In [3]:
input_data_path = "./data/raw_data/uci-default-of-credit-card/data/data.csv"
local_path_save = './data/mlflow_artifacts/'

In [4]:
fitted_ohe = load(open(local_path_save + '_ohe.pkl', 'rb')) 
fitted_scaler = load(open(local_path_save + '_scaler.pkl', 'rb'))

/Users/davideposillipo/.pyenv/versions/3.10.9/envs/prova/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator OneHotEncoder from version 1.2.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/davideposillipo/.pyenv/versions/3.10.9/envs/prova/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator StandardScaler from version 1.2.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [5]:
pp = Preprocessing(input_data_path, "default")
X, Y = pp.read_dataframe()

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.3, random_state=420)

X_test_ohe, _, _ = pp.preprocess_for_classification(df=X_test, 
                                                fit_ohe=True, 
                                                fitted_ohe=fitted_ohe,
                                                perform_scaling=True,
                                                fitted_scaler=fitted_scaler)

In [6]:
with open("notebooks/mlruns/1/1e4a0667c7a64cbe8c7b023410e5781c/artifacts/model/model.pkl", "rb") as file:
    classifier = load(file)

In [7]:
predicted_prob = classifier.predict_proba(X_test_ohe)
predicted_values = classifier.predict(X_test_ohe)

#### Definition of conditioning variables

In [8]:
def age_buckets(x):
    if x < 30:
        return 1
    elif x < 40:
        return 2
    else:
        return 3

X_test['age_buckets'] = X.x5_age.apply(age_buckets)

In [9]:
conditioning_variables = ['x3_education', 'x4_marriage', 'age_buckets']

In [10]:
df_with_predictions = pd.concat(
    [X_test.reset_index(drop=True), pd.Series(predicted_values)], axis=1).rename(columns={0:"predictions"})

## Hazard and risk functions

In [11]:
def hazard_function(overall_result, conditioned_results, tot_observations):
    
    # test result, threshold, num_samples, boolean
    test_results = []
    test_results.append((overall_result[0], 
                    overall_result[2], 
                    tot_observations, 
                    overall_result[1]))
    
    for group in conditioned_results.values():
        if (group[1] is not None):
            test_results.append((group[1], group[3], group[0], group[2]))
    
    hazard = 0
    for line in test_results:
        weight = 1 #to be implemented
        delta = 1 if line[3]==False else 0
        q = line[2]/tot_observations
        e = line[0] - line[1]
        hazard += delta * weight * q * e
        
    average_threshold = np.mean([x[1] for x in test_results])
        
    return hazard, average_threshold

In [12]:
def risk_function(test_hazards, average_thresholds):
    # test_hazards = [list_of_hazards]
    # average_thresholds = [mean(thresholds_of_test1), mean(thresholds_of_a_test2), ...], 
    #    needed if automatic threshold is used
    risk = 0
    for hazard, threshold in zip(test_hazards, average_thresholds):
        risk += hazard * threshold
        
    risk = risk/len(test_hazards)**2
    
    return risk

### Test 1: TVD, A1=high

In [13]:
bd_1 = FreqVsFreqBiasDetector(distance="TVD", A1="high")

In [14]:
overall_1 = bd_1.compare_root_variable_groups(
    dataframe=df_with_predictions,
    target_variable='predictions',
    root_variable='x2_sex')

In [15]:
conditioned_1 = bd_1.compare_root_variable_conditioned_groups(
    dataframe=df_with_predictions,
    target_variable='predictions',
    root_variable='x2_sex',
    conditioning_variables=conditioning_variables)

In [16]:
hazard_test_1, average_threshold_1 = hazard_function(
    overall_1, 
    conditioned_1, 
    df_with_predictions.shape[0])

### Test 2 (TVD, low)

In [17]:
bd_2 = FreqVsFreqBiasDetector(distance="TVD", A1="low")

In [18]:
overall_2 = bd_2.compare_root_variable_groups(
    dataframe=df_with_predictions,
    target_variable='predictions',
    root_variable='x2_sex')

In [19]:
conditioned_2 = bd_2.compare_root_variable_conditioned_groups(
    dataframe=df_with_predictions,
    target_variable='predictions',
    root_variable='x2_sex',
    conditioning_variables=conditioning_variables)

In [20]:
hazard_test_2, average_threshold_2 = hazard_function(
    overall_2, 
    conditioned_2, 
    df_with_predictions.shape[0])

### Test 3 (JS, high)

In [21]:
bd_3 = FreqVsFreqBiasDetector(distance="JS", A1="high")

In [22]:
overall_3 = bd_3.compare_root_variable_groups(
    dataframe=df_with_predictions,
    target_variable='predictions',
    root_variable='x2_sex')

In [23]:
conditioned_3 = bd_3.compare_root_variable_conditioned_groups(
    dataframe=df_with_predictions,
    target_variable='predictions',
    root_variable='x2_sex',
    conditioning_variables=conditioning_variables)

In [24]:
hazard_test_3, average_threshold_3 = hazard_function(
    overall_3, 
    conditioned_3, 
    df_with_predictions.shape[0])

### Test 4 (JS, low)

In [25]:
bd_4 = FreqVsFreqBiasDetector(distance="JS", A1="low")

In [26]:
overall_4 = bd_4.compare_root_variable_groups(
    dataframe=df_with_predictions,
    target_variable='predictions',
    root_variable='x2_sex')

In [27]:
conditioned_4 = bd_4.compare_root_variable_conditioned_groups(
    dataframe=df_with_predictions,
    target_variable='predictions',
    root_variable='x2_sex',
    conditioning_variables=conditioning_variables)

In [28]:
hazard_test_4, average_threshold_4 = hazard_function(
    overall_4, 
    conditioned_4, 
    df_with_predictions.shape[0])

## Risk results

In [29]:
hazards = [hazard_test_1, hazard_test_2, hazard_test_3, hazard_test_4]
average_thresholds = [average_threshold_1, average_threshold_2, average_threshold_3, average_threshold_4]

In [30]:
risk_function(hazards, average_thresholds)

0.00015458111237839916